# Jane Street: TabNet  + PurgedGroupKFold

* Reference:
    * Data process: https://www.kaggle.com/snippsy/bottleneck-encoder-mlp-keras-tuner
    * Fast fill NA: https://www.kaggle.com/gogo827jz/optimise-speed-of-filling-nan-function
    * Label smoothing: https://arxiv.org/abs/1906.02629


In [ ]:
import pickle
import numpy  as np
import pandas as pd
import datatable as dt
from glob import glob

!pip install  ../input/officialpytorchtabnet/pytorch_tabnet-3.0.0-py3-none-any.whl pytorch-tabnet  > /dev/null
from pytorch_tabnet.tab_model import TabNetClassifier

from tqdm.notebook import tqdm

## PurgedGroupTimeSeriesSplit

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
import random, torch, os

TRAINING = True
SEED = 777
MAX_EPOCHS = 200
BATCH_SIZE = 4096
VIRTUAL_BATCH_SIZE = 256
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## Loading the training data

In [ ]:
if TRAINING:
    train_data = dt.fread('../input/jane-street-market-prediction/train.csv').to_pandas()
    train_data = train_data.query('date > 85').reset_index(drop = True) 
    train_data = train_data.astype({c: np.float32 for c in train_data.select_dtypes(include='float64').columns}) #limit memory use
    train_data.fillna(train_data.mean(),inplace=True)
    train_data = train_data.query('weight > 0').reset_index(drop = True)
    train_data['action'] = (train_data['resp'] > 0).astype('int')
    features = [c for c in train_data.columns if 'feature' in c]    
    
    X = train_data[features].values
    y = train_data['action'].values
    f_mean = np.mean(train_data[features[1:]].values,axis=0)
    np.save('f_mean.npy',f_mean)    
else:
    train_data = pd.read_csv('../input/jane-street-market-prediction/train.csv',nrows=10)
    features = [c for c in train_data.columns if 'feature' in c]    
    f_mean = np.load('../input/tabnettest1/f_mean.npy')

> ## QuantileTransformer

In [ ]:
from sklearn.preprocessing import QuantileTransformer
if TRAINING:
    for col in tqdm(range(X.shape[1])):    
        Xt = X[:,col].reshape(X.shape[0], 1)
        transformer = QuantileTransformer(n_quantiles=100, random_state=0, 
                                          output_distribution="normal")
        transformer.fit(Xt)
        pd.to_pickle(transformer, f'quantile_transformer_{col}.pkl')
        X[:,col] = transformer.transform(X[:,col].reshape(X.shape[0], 1)).reshape(-1)

In [ ]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
seed_everything(SEED)

## Train TabNetClassifier on GPU

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self):
        super(LabelSmoothingCrossEntropy, self).__init__()
        
    def forward(self, x, target, smoothing=0.01):
        confidence = 1. - smoothing
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = confidence * nll_loss + smoothing * smooth_loss
        return loss.mean()
    
loss_fn = LabelSmoothingCrossEntropy()

In [ ]:
clf = TabNetClassifier(n_d=64, 
                       n_a=64, 
                       n_steps=1,
                       gamma=1.5, 
                       lambda_sparse=1e-4, 
                       momentum=0.3, 
                       clip_value=2., 
                       mask_type='entmax',
                       optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=1e-4, weight_decay=1e-5),
                       scheduler_params=dict(milestones=[100, 150], gamma=0.9),
                       scheduler_fn=torch.optim.lr_scheduler.MultiStepLR,
                       epsilon=1e-15,
                       verbose=50,
                       seed=SEED,
                       device_name = DEVICE
)

## 5 folds CV
* Only the last two folds are used due to the performance of public lb on experience and inference time limit, maybe overfit on private

In [ ]:
if TRAINING:
    
    from sklearn.metrics import roc_auc_score

    FOLDS = 5
    gkf = PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap=20)
    splits = list(gkf.split(y, groups=train_data['date'].values))
    
    oof_score = 0
    
    for fold, (train_indices, test_indices) in enumerate(splits[-2:]):
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]

        clf.fit(X_train=X_train, y_train=y_train,
                eval_set=[(X_test, y_test)],
                eval_metric = ['auc'],
                max_epochs=MAX_EPOCHS , 
                patience=20,
                batch_size=BATCH_SIZE, 
                virtual_batch_size=VIRTUAL_BATCH_SIZE,
                num_workers=0,
                weights=1,
                drop_last=False,
                loss_fn=loss_fn
        )
        
        with open(f"tabnet_SEED{SEED}_FOLD{fold}.pkl", 'wb') as model_file:
            pickle.dump(clf, model_file)        
        
        valid_preds = clf.predict_proba(X_test)[:,1]
        oof = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        oof_score += oof/FOLDS 
        print(' ')
        #print(f'Fold: {fold+1}, AUC: {oof:.5f}')
    print(f"OOF AVG: {oof_score:.5f}")
else:
    import pickle
    models = []
    for file in glob('../input/tabnettest1/*.pkl'):
        with open(file,"rb") as model_file:
            _clf = pickle.load(model_file)
        models.append(_clf)

## Fast Filling-NaN Function

In [ ]:
import numpy as np
from numba import njit

@njit
def fillna_npwhere_njit(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

def for_loop(method, matrix, values):
    for i in range(matrix.shape[0]):
        matrix[i] = method(matrix[i], values)
    return matrix

## Submission

In [ ]:
def trans(value):
    for col in range(130):
        transformer = pd.read_pickle(f'quantile_transformer_{col}.pkl')  
        value[0][col] = transformer.transform(value[:,col].reshape(1,1))[0]
    return value

In [ ]:
import janestreet
env = janestreet.make_env()

if not TRAINING:
    f = np.median
    opt_th = 0.5
    for (test_df, pred_df) in tqdm(env.iter_test()):
        if test_df['weight'].item() > 0:
            x_tt = test_df.loc[:, features].values
            x_tt[:, 1:] = for_loop(fillna_npwhere_njit, x_tt[:, 1:], f_mean)
            trans(x_tt)
            pred=0.
            for model in models:
                pred += model.predict_proba(x_tt)[0][1] / len(models)
            pred_df.action = np.where( f(pred) >= opt_th, 1, 0).astype(int)
        else:
            pred_df.action = 0
        env.predict(pred_df)